---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [7]:
import dateutil.parser as dparser

sorted_dates = []

count = 0

for i in range(len(df)):
    
    try:
        ans = dparser.parse(df[i],fuzzy=True,)
        sorted_dates.append(ans)
    
    except:
        
        ans = date_parse1(df[i])
        sorted_dates.append(ans)
        count += 1
                
print(count)

2027-07-09 00:00:00
1996-02-06 00:00:00
2018-04-05 00:00:00
1989-10-24 00:00:00
1985-05-11 00:00:00
1990-05-24 00:00:00
2011-01-25 00:00:00
2013-01-10 00:00:00
2024-04-04 00:00:00
2059-05-21 00:00:00
1990-03-03 00:00:00
1976-02-11 00:00:00
1984-07-25 00:00:00
1989-09-22 00:00:00
1976-09-02 00:00:00
1972-07-20 00:00:00
1987-04-12 00:00:00
1973-02-14 00:00:00
1988-05-24 00:00:00
1975-07-29 00:00:00
1987-06-24 00:00:00
1982-08-16 00:00:00
1991-07-15 00:00:00
1992-06-13 00:00:00
1986-12-26 00:00:00
1987-10-11 00:00:00
1973-12-01 00:00:00
2000-01-01 00:00:00
1983-08-06 00:00:00
1994-07-29 00:00:00
1978-08-11 00:00:00
2003-11-24 00:00:00
1982-07-04 00:00:00
2000-01-01 00:00:00
1981-09-19 00:00:00
2006-10-15 00:00:00
2004-02-08 00:00:00
2000-09-14 00:00:00
1993-06-05 00:00:00
0100-02-02 00:00:00
1989-08-26 00:00:00
1995-10-13 00:00:00
1976-09-20 00:00:00
2019-11-14 00:00:00
1997-10-05 00:00:00
2002-07-18 00:00:00
1978-02-03 00:00:00
2006-02-11 00:00:00
2019-09-21 00:00:00
1992-12-15 00:00:00


In [8]:
df_sort = sorted(sorted_dates)

TypeError: '<' not supported between instances of 'NoneType' and 'datetime.datetime'

In [9]:
df[0][1:]

'3/25/93 Total time of visit (in minutes):\n'

In [5]:
def date_parse2(text):

    try:

        ans = dparser.parse(text, fuzzy=True)
        print(ans)

    except:
        if len(text) == 1:
            print("2000-01-01 00:00:00")
        else:
            text = text[1:]
            date_parse2(text)

def date_parse1(text):

    try:

        ans = dparser.parse(text, fuzzy=True)
        print(ans)

    except:
        
        if len(text) == 1:
            date_parse2(text)
        else:
            text = text[:-1]
            date_parse1(text)

In [10]:
df[4]

'2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n'

In [12]:
a = date_parse1(df[4])
a

1996-02-06 00:00:00


In [13]:
b ="a"
len(b)

1

In [15]:
import re

In [21]:
text8 = df.iloc[0]
text8

'03/25/93 Total time of visit (in minutes):\n'

In [24]:
[w for w in text8 if re.search('[{2}/{2}/{2}]', w)]

['/', '2', '/']

In [ ]:
# 04/20/2009; 04/20/09; 4/20/09; 4/3/09
# Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
# 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
# Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
# Feb 2009; Sep 2009; Oct 2010
# 6/2008; 12/2009
# 2009; 2010

In [ ]:
#import regex
import re

#import calendar
import calendar

short_month = {v: k for k,v in enumerate(calendar.month_abbr)}

# Import the datetime module 
from datetime import datetime 

In [293]:
#putting it all together

date_list = []

#find all dates of type month/day/year


for i in dates:
    
    try:
        m = re.search('\d+(/)\d+(/)\d+', i)
        date = m.group(0)

        date = date.split("/")

        month = int(date[0])

        year = int(date[2][-2:])

        day = int(date[1])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)

    except:
        continue
#find all dates of type day, name of month, year



    try: 
        m = re.search('\d+ (.+) \d+', i)
        found1 = m.group(1)
        month = found1.lower()
        month = month.title()

        month = int(short_month[month[:3]])

        m = re.search('(\d+) .+ \d+', i, re.IGNORECASE)
        found2 = m.group(1)

        day = int(found2)

        m = re.search('\d+ .+ (\d+)', i, re.IGNORECASE)
        found3 = m.group(1)

        year = int(found3[-2:])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
    
    except:
        continue
    
    
#find all dates of type name month, day, year except month-day-year


    try:

        m = re.search('(.+) \d+', i)
        found1 = m.group(1)
        month = found1.lower()
        month = month.title()  
        month = int(short_month[month[:3]])

        m = re.search('(.+) \d+', i)
        found2 = m.group(1)
        day = found2
        m1 = re.search('.+ (\d+)', day)
        found3 = m1.group(1)
        day = int(found3)

        m = re.search('.+ (\d+)', i)
        found4 = m.group(1)
        year = int(found4[-2:])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
    
    except:
        continue
        
        
#find all dates of type month-day-year

    try:

        m = re.search('\w+(-)\d+(-)\d+', i)
        date = m.group(0)
        date = date.split("-")

        month = date[0].lower()
        month = month.title()  
        month = int(short_month[month[:3]])

        day = int(date[1])

        year = int(date[2][-2:])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
    
    except:
        continue
    
#find all dates with only name month and year

    try:

        m = re.search('.+ (\d+)', i)
        found1 = m.group(1)
        year = found1

        m = re.search(f'(.+) {year}', i)
        found2 = m.group(1)
        month = found2
        month = month.lower()
        month = month.title()  
        month = int(short_month[month[:3]])

        day = 1

        year = int(year[-2:])

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)

    except:
        continue

#find all dates with month/year


    try:

        m = re.search('\d+(/)\d+', i)
        date = m.group(0)

        date = date.split("/")

        month = int(date[0])

        year = int(date[1][-2:])

        day = 1

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)

    except:
        continue
        
#find all dates with year

    try:

        word_list = i.split(" ")

        for a in word_list:

            if a.isalpha():
                continue

            else:
                year = a

        year = int(year[-2:])

        month = 1

        day = 1

        date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
        date = datetime.strptime(date, '%m/%d/%y')

        date_list.append(date)
    
    except:
        continue

In [294]:
date_list

[datetime.datetime(2009, 4, 20, 0, 0),
 datetime.datetime(2009, 4, 20, 0, 0),
 datetime.datetime(2009, 4, 20, 0, 0),
 datetime.datetime(2009, 4, 3, 0, 0)]

In [291]:
#testing       
dates1 =  ["23 Jun 2018", "2 Dec 2017", "11 Jun 2018", 
        "01 January 2019", "10 mar. 15", "01 Jan 2007"] 

dates2 = ["asdfa 04/20/2009 asdf", "04/20/09", "4/20/09", "4/3/09"]

dates3= ['Mar 20, 2009', 'March 20, 2009', 'Mar. 20, 2009', 'Mar 20 2009']

dates4 =  ["asdf Mar-20-2009 asdf", " sadf Feb-01-2015 asdf"]

dates5 = ["Mar 20th, 2009", "Mar 21st, 2009", "Mar 22nd, 2009"]

dates6 = ["Feb 2009", "Sep 2009", "Oct 2010"]

dates7 = ["6/2008", "12/2009"]

dates8 = ["asdf 2008 asdfasd", " as 2017 asad"]

dates = dates1+dates2+dates3+dates4+dates5+dates6+dates7+dates8

In [288]:
for i in range(0,10):
    
    print(i)
    try:
        if i == 1:
            print("dog")
    except:
        continue
    try:
        if i == 4:
            print("cat")
    except:
        continue
        

0
1
dog
2
3
4
cat
5
6
7
8
9


In [255]:
i = dates[0]

m = re.search('\w+(-)\d+(-)\d+', i)
date = m.group(0)
date

'Mar-20-2009'

In [261]:
a = dates[1]
a

'12/2009'

In [266]:
m = re.search('\d+(/)\d+', a)
found1 = m.group(0)
found1

'12/2009'

In [202]:
#find all dates with year

date_list = []

for i in dates:
    
    word_list = i.split(" ")
    
    for a in word_list:
        
        if a.isalpha():
            continue
        
        else:
            year = a
    
    year = int(year[-2:])
    
    month = 1
    
    day = 1
        
    date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
    date = datetime.strptime(date, '%m/%d/%y')

    date_list.append(date)
        
date_list          
    


[datetime.datetime(2008, 1, 1, 0, 0), datetime.datetime(2017, 1, 1, 0, 0)]

In [268]:
#find all dates with month/year

date_list = []

for i in dates:
    
    m = re.search('\d+(/)\d+', i)
    date = m.group(0)
    
    date = date.split("/")
    
    month = int(date[0])
    
    year = int(date[1][-2:])
    
    day = 1
    
    date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
    date = datetime.strptime(date, '%m/%d/%y')

    date_list.append(date)
        
date_list       

[datetime.datetime(2008, 6, 1, 0, 0), datetime.datetime(2009, 12, 1, 0, 0)]

In [193]:
#find all dates with only name month and year

date_list = []

for i in dates:
    
    m = re.search('.+ (\d+)', i)
    found1 = m.group(1)
    year = found1
    
    m = re.search(f'(.+) {year}', i)
    found2 = m.group(1)
    month = found2
    month = month.lower()
    month = month.title()  
    month = int(short_month[month[:3]])
    
    day = 1
    
    year = int(year[-2:])
    
    date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
    date = datetime.strptime(date, '%m/%d/%y')

    date_list.append(date)
        
date_list       

[datetime.datetime(2009, 2, 1, 0, 0),
 datetime.datetime(2009, 9, 1, 0, 0),
 datetime.datetime(2010, 10, 1, 0, 0)]

In [176]:
#find all dates of type month-day-year

date_list = []

for i in dates:
    
    date = i.split("-")
    
    month = date[0].lower()
    month = month.title()  
    month = int(short_month[month[:3]])
    
    day = int(date[1])
    
    year = int(date[2][-2:])
    
    date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
    date = datetime.strptime(date, '%m/%d/%y')

    date_list.append(date)

        
date_list       

[datetime.datetime(2009, 3, 20, 0, 0), datetime.datetime(2015, 2, 1, 0, 0)]

In [179]:
#find all dates of type name month, day, year except month-day-year

date_list = []

for i in dates:

    m = re.search('(.+) \d+', i)
    found1 = m.group(1)
    month = found1.lower()
    month = month.title()  
    month = int(short_month[month[:3]])

    m = re.search('(.+) \d+', i)
    found2 = m.group(1)
    day = found2
    m1 = re.search('.+ (\d+)', day)
    found3 = m1.group(1)
    day = int(found3)

    m = re.search('.+ (\d+)', i)
    found4 = m.group(1)
    year = int(found4[-2:])

    date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
    date = datetime.strptime(date, '%m/%d/%y')

    date_list.append(date)

        
date_list    

[datetime.datetime(2009, 3, 20, 0, 0),
 datetime.datetime(2009, 3, 21, 0, 0),
 datetime.datetime(2009, 3, 22, 0, 0)]

In [94]:
#find all dates of type day, name of month, year

date_list = []

for i in dates:

    m = re.search('\d+ (.+) \d+', i)
    found1 = m.group(1)
    month = found1.lower()
    month = month.title()
    
    month = int(short_month[month[:3]])

    m = re.search('(\d+) .+ \d+', i, re.IGNORECASE)
    found2 = m.group(1)

    day = int(found2)

    m = re.search('\d+ .+ (\d+)', i, re.IGNORECASE)
    found3 = m.group(1)

    year = int(found3[-2:])
        
    date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
    date = datetime.strptime(date, '%m/%d/%y')
    
    date_list.append(date)


#sort all dates 

date_list = sorted(date_list)

#converts date.time to mm/dd/yy

date_final = []

for i in date_list:
    
    month = f'{i.month:02d}'
    
    day = f'{i.day:02d}'
    
    year = str(i.year)[-2:]
    year = f'{year}'
    
    date = month + "/" + day + "/" + year
    
    date_final.append(date)

date_final


['01/01/07', '03/10/15', '12/02/17', '06/11/18', '06/23/18', '01/01/19']

In [ ]:
#find all dates of type day, name of month, year

date_list = []

for i in dates:

    m = re.search('\d+ (.+) \d+', i)
    found1 = m.group(1)
    month = found1.lower()
    month = month.title()
    
    month = int(short_month[month[:3]])

    m = re.search('(\d+) .+ \d+', i, re.IGNORECASE)
    found2 = m.group(1)

    day = int(found2)

    m = re.search('\d+ .+ (\d+)', i, re.IGNORECASE)
    found3 = m.group(1)

    year = int(found3[-2:])
        
    date = f'{month:02d}' + '/' + f'{day:02d}' + '/' + f'{year:02d}'
    date = datetime.strptime(date, '%m/%d/%y')
    
    date_list.append(date)
    
    

In [77]:
sorted(date_list)

[datetime.datetime(2007, 1, 1, 0, 0),
 datetime.datetime(2015, 3, 10, 0, 0),
 datetime.datetime(2017, 12, 2, 0, 0),
 datetime.datetime(2018, 6, 11, 0, 0),
 datetime.datetime(2018, 6, 23, 0, 0),
 datetime.datetime(2019, 1, 1, 0, 0)]

In [91]:
str(date_list[0].year)[-2:]

'07'

In [23]:
for i in dates:

    m = re.search('\d+ \w+ (\d+)', i, re.IGNORECASE)
    found = m.group(1)

    print(found)

2018
2017
2018
2019
2016
2007


In [70]:
short_month[found]

1

In [39]:
import calendar

short_month = {v: k for k,v in enumerate(calendar.month_abbr)}


In [45]:
short_month["Jun"]

6

In [14]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here